<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->



# **P1 - RA PAR**

In [1]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Enzo Sakamoto, 21002100' #@param {type:"string"}






In [2]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804 entries, 0 to 803
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              804 non-null    int64  
 1   name                            803 non-null    object 
 2   host_id                         804 non-null    int64  
 3   host_name                       804 non-null    object 
 4   neighbourhood_group             804 non-null    object 
 5   neighbourhood                   804 non-null    object 
 6   latitude                        804 non-null    float64
 7   longitude                       804 non-null    float64
 8   room_type                       804 non-null    object 
 9   price                           804 non-null    int64  
 10  minimum_nights                  804 non-null    int64  
 11  number_of_reviews               804 non-null    int64  
 12  last_review                     628 

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [121]:
df.groupby('host_id').agg(
    count=('host_id','count')
).sort_values('count',ascending=False).head(3)

count
host_id         
219517861      7
30283594       4
120762452      4

In [115]:
top_hosts = df.groupby('host_id').agg(
    count=('host_id','count')
).sort_values('count',ascending=False).head(3).reset_index().host_id.tolist()

top_hosts

[219517861, 30283594, 120762452]

In [116]:
df[df.host_id.isin(top_hosts)][['host_id', 'host_name']].drop_duplicates()

host_id     host_name
241   30283594          Kara
402  120762452       Stanley
676  219517861  Sonder (NYC)

# Q2A. Locatários profissionais $\times$ anunciantes com um único imóvel

Considere o preço médio dos imóveis. Qual a diferença do preço médio dos imóveis dos anunciantes com mais de um imóvel e dos anunciantes com um único imóvel anunciado na plataforma?

In [28]:
df_greater_1 = df.groupby('host_id').agg(
    total=('host_id','count')
).reset_index().sort_values('total',ascending=False).copy()

df_greater_1 = df_greater_1[df_greater_1.total > 1]['host_id']

In [29]:
df_equals_1 = df.groupby('host_id').agg(
    total=('host_id','count')
).reset_index().sort_values('total',ascending=False).copy()

df_equals_1 = df_equals_1[df_equals_1.total == 1]['host_id']

In [122]:
preco_mais_de_um_imovel = df[df['host_id'].isin(df_greater_1)].price.sum()
preco_mais_de_um_imovel

np.int64(9381)

In [123]:
preco_mais_um_imovel = df[df['host_id'].isin(df_equals_1)].price.sum()
preco_mais_um_imovel

np.int64(114416)

In [124]:
...

print( f'Diferença de preços: USD {preco_mais_de_um_imovel - preco_mais_um_imovel:.2f}')

Diferença de preços: USD -105035.00


In [125]:
len(df_greater_1)

16

In [126]:
len(df_equals_1)

761

# Q3A.  Manhattan *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [37]:
def identify_outliers(data: pd.Series):
  Q1 = data.quantile(0.25)
  Q3 = data.quantile(0.75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR
  outliers = data[(data < lower_bound) | (data > upper_bound)].index.tolist()
  return outliers

In [127]:
identify_outliers(df[df.neighbourhood_group == 'Manhattan'].price)

[23,
 35,
 54,
 82,
 97,
 150,
 151,
 209,
 297,
 308,
 323,
 339,
 365,
 499,
 510,
 563,
 623,
 645,
 677,
 680,
 684,
 707,
 708,
 718,
 727,
 738,
 795]

In [46]:
perc_Manhattan = len(identify_outliers(df[df.neighbourhood_group == 'Manhattan'].price)) / len(df[df.neighbourhood_group == 'Manhattan'].price) * 100

In [128]:
...

print( f'Percentuais de outliers de preços, Manhattan: {perc_Manhattan:.2f} %')


Percentuais de outliers de preços, Manhattan: 7.34 %


# Q4A. Preços $\times$ Descrições

Considere as descrições dos anúncios em `df.name`. Crie uma coluna `df['word']` que contém 'beautiful','sunny','spacious' ou 'heart' de acordo com a presença dessa palavra no anúncio. Caso não tenha nenhuma das palavras, assinale 'others'. Ao final, verifique a quantidade e o preço médio dos imóveis para cada grupo dessas palavras nos anúncios.

Atenção, considere a ordem 'beautiful','sunny','spacious' ou 'heart' (isto é, se o texto contém duas ou mais palavras, por exemplo 'beautiful' e 'sunny', considere 'beautiful'). Considere também quaisquer combinações de maiúsculas/minúsculas nas palavras.

In [129]:
df_word = df.copy()

df_word['name'] = df_word['name'].str.lower()

df_word['word'] = df_word['name'].apply(
    lambda x: 'beautiful' if 'beautiful' in str(x) else
              'sunny' if 'sunny' in str(x) else
              'spacious' if 'spacious' in str(x) else
              'heart' if 'heart' in str(x) else
              'others'
)

df_word[df_word.word != 'others'].head(10)

id                                            name  host_id  \
6    233662              large sunny luxe prvt room/midtown  1223568   
11   292266                      sunny and spacious bedroom  1513294   
16   435909                        sunny west village dream  2165401   
17   482765                   beautiful brooklyn brownstone  2389885   
19   571564                             large spacious room  1141935   
33   832596                spacious 1 br near the water ues  1723222   
36   944049             beautiful 1br in prime west village  1908073   
37   969463  artsy fashion pad in the heart of williamsburg  1447988   
40  1354973              gorgeous spacious. 2 subways close   306394   
47  1944891                  spacious one bedroom apartment   861330   

           host_name neighbourhood_group       neighbourhood  latitude  \
6            Tiffany           Manhattan             Midtown  40.75575   
11           Urszula            Brooklyn            Bushwick  40.69755   
16            Andrew           Manhattan        West Village  40.73775   
17             Megan            Brooklyn         Fort Greene  40.68834   
19               Ana            Brooklyn       Crown Heights  40.67436   
33              Nick           Manhattan     Upper East Side  40.77613   
36            Andrew           Manhattan        West Village  40.73194   
37  Patrick And Erin            Brooklyn        Williamsburg  40.71916   
40        Marialuisa            Brooklyn  Bedford-Stuyvesant  40.68420   
47            Joseph            Brooklyn        Williamsburg  40.70139   

    longitude        room_type  price  minimum_nights  number_of_reviews  \
6   -73.96842     Private room    110               2                 61   
11  -73.91187     Private room     50              60                 17   
16  -74.00344  Entire home/apt    151              88                 19   
17  -73.97875     Private room    100               2                112   
19  -73.94807     Private room     95               1                123   
33  -73.94872  Entire home/apt    250               4                  1   
36  -74.00189  Entire home/apt    200               4                 45   
37  -73.95696  Entire home/apt    150               2                 11   
40  -73.91770     Private room     59               1                123   
47  -73.94499  Entire home/apt     49               7                  0   

   last_review  reviews_per_month  calculated_host_listings_count  \
6   2019-05-18               0.65                               1   
11  2019-04-01               0.19                               2   
16  2016-12-24               0.22                               1   
17  2019-06-30               1.30                               2   
19  2019-06-02               1.47                               3   
33  2016-05-24               0.03                               1   
36  2019-07-01               0.59                               1   
37  2018-10-15               0.14                               1   
40  2019-06-30               2.23                               1   
47         NaN                NaN                               3   

    availability_365       word  
6                264      sunny  
11                74      sunny  
16               249      sunny  
17                12  beautiful  
19               359   spacious  
33                 0   spacious  
36                48  beautiful  
37               364      heart  
40               336   spacious  
47                 0   spacious

In [130]:
df_word.groupby('word').agg(
    total=('word','count'),
    price_mean=('price','mean')
).sort_values('price_mean',ascending=False)

total  price_mean
word                        
heart         36  166.388889
others       624  157.886218
beautiful     41  137.926829
sunny         43  137.674419
spacious      60  128.516667

# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [132]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes

id   type   value
0     35907127  taxes  1315.0
1       804815  taxes     NaN
2     22328585  taxes   900.0
3     27876600  taxes   869.0
4     34976037  taxes  1629.0
...        ...    ...     ...
1523  34284316  fines     NaN
1524  20325804  fines     NaN
1525  25872986  fines     NaN
1526  19365218  fines   906.0
1527   4235843  fines     NaN

[1528 rows x 3 columns]

In [133]:
df_with_taxes = df.merge(df_taxes, how='left', on='id')
df_with_taxes['value'] = df_with_taxes['value'].replace(np.nan, 0)
df_with_taxes.head()

id                                            name  host_id   host_name  \
0  20913              Charming 1 bed GR8 WBurg LOCATION!    79402  Christiana   
1  20913              Charming 1 bed GR8 WBurg LOCATION!    79402  Christiana   
2  25235               Large 2 Bedroom Great for Groups!    87773      Shelly   
3  25235               Large 2 Bedroom Great for Groups!    87773      Shelly   
4  76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971      Evelyn   

  neighbourhood_group       neighbourhood  latitude  longitude  \
0            Brooklyn        Williamsburg  40.70984  -73.95775   
1            Brooklyn        Williamsburg  40.70984  -73.95775   
2            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
3            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
4            Brooklyn         South Slope  40.66552  -73.99019   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    100               5                168  2018-07-22   
2  Entire home/apt    125              90                162  2019-06-28   
3  Entire home/apt    125              90                162  2019-06-28   
4  Entire home/apt    169               2                398  2019-06-28   

   reviews_per_month  calculated_host_listings_count  availability_365   type  \
0               1.57                               1                 0  taxes   
1               1.57                               1                 0  fines   
2               1.46                               2               137  taxes   
3               1.46                               2               137  fines   
4               3.97                               3               182  taxes   

    value  
0   789.0  
1   796.0  
2  1779.0  
3     0.0  
4  1516.0

In [136]:
df_taxes_reset = df_with_taxes.groupby(['neighbourhood_group', 'type']).agg(
    value_mean=('value','mean')
).reset_index()

df_taxes_reset['value_mean'] = df_taxes_reset['value_mean'].astype(int)

df_taxes_reset

neighbourhood_group   type  value_mean
0               Bronx  fines         126
1               Bronx  taxes         851
2            Brooklyn  fines         314
3            Brooklyn  taxes         858
4           Manhattan  fines         314
5           Manhattan  taxes         888
6              Queens  fines         322
7              Queens  taxes         804
8       Staten Island  fines         581
9       Staten Island  taxes        1157

# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [142]:
sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,28]) ].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

A soma dos preços das páginas 1, 24, 25 e 28 é: 12633


#### Aqui um *sample* do report.

In [ ]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price  page_id  line_id
795              Queens       Woodside   75.000000       28       17
796              Queens       Woodside  120.000000       28       18
797              Queens           MEAN   92.227848       28       19
798       Staten Island  Randall Manor   99.000000       29        1
799       Staten Island    South Beach   20.000000       29        2
800       Staten Island    South Beach  100.000000       29        3
801       Staten Island     St. George  130.000000       29        4
802       Staten Island      Todt Hill  135.000000       29        5
803       Staten Island  Tompkinsville   59.000000       29        6
804       Staten Island           MEAN   89.428571       29        7

In [139]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price  page_id  line_id
799              Queens       Woodside  120.000000       29       19
800              Queens           MEAN   92.227848       29       20
801       Staten Island  Arden Heights   83.000000       30        1
802       Staten Island  Randall Manor   99.000000       30        2
803       Staten Island    South Beach   20.000000       30        3
804       Staten Island    South Beach  100.000000       30        4
805       Staten Island     St. George  130.000000       30        5
806       Staten Island      Todt Hill  135.000000       30        6
807       Staten Island  Tompkinsville   59.000000       30        7
808       Staten Island           MEAN   89.428571       30        8

In [100]:
df_aux = df[['neighbourhood_group', 'neighbourhood', 'price']].copy().sort_values(['neighbourhood_group', 'neighbourhood'])

df_aux.price = df_aux.price.astype(float)

neighbourhood_groups = df_aux.neighbourhood_group.unique()

df_report = pd.DataFrame()

for ng in neighbourhood_groups:
  df_filtered = df_aux.loc[df.neighbourhood_group == ng].copy()
  mean = df_filtered.price.mean()
  df_filtered = pd.concat([
      df_filtered,
      pd.DataFrame(
          [
            {
                'neighbourhood_group': ng,
                'neighbourhood': 'MEAN',
                'price': mean
            }
          ]
      )
  ], ignore_index=True)
  df_report = pd.concat([
      df_report,
      df_filtered
  ], ignore_index=True)

page = 1
count = 0
df_report['page_id'] = 0
for i in df_report.index:
  if df_report.loc[i].neighbourhood == 'MEAN':
    df_report.loc[i, 'page_id'] = page
    page += 1
    count = 0
  else:
    count += 1
    df_report.loc[i, 'page_id'] = page
    if count % 30 == 0:
      page += 1
      count = 0

init = 1
actual_page = df_report.loc[0].page_id
df_report['line_id'] = 0
for i in df_report.index:
  if df_report.loc[i].page_id > actual_page:
    actual_page = df_report.loc[i].page_id
    init = 1
  df_report.loc[i, 'line_id'] = init
  init += 1

df_report

neighbourhood_group      neighbourhood       price  page_id  line_id
0                 Bronx            Belmont   29.000000        1        1
1                 Bronx  Claremont Village   67.000000        1        2
2                 Bronx          Concourse   40.000000        1        3
3                 Bronx           Edenwald   90.000000        1        4
4                 Bronx           Edenwald   80.000000        1        5
..                  ...                ...         ...      ...      ...
804       Staten Island        South Beach  100.000000       30        4
805       Staten Island         St. George  130.000000       30        5
806       Staten Island          Todt Hill  135.000000       30        6
807       Staten Island      Tompkinsville   59.000000       30        7
808       Staten Island               MEAN   89.428571       30        8

[809 rows x 5 columns]

In [104]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price  page_id  line_id
799              Queens       Woodside  120.000000       29       19
800              Queens           MEAN   92.227848       29       20
801       Staten Island  Arden Heights   83.000000       30        1
802       Staten Island  Randall Manor   99.000000       30        2
803       Staten Island    South Beach   20.000000       30        3
804       Staten Island    South Beach  100.000000       30        4
805       Staten Island     St. George  130.000000       30        5
806       Staten Island      Todt Hill  135.000000       30        6
807       Staten Island  Tompkinsville   59.000000       30        7
808       Staten Island           MEAN   89.428571       30        8

In [103]:
df_report

neighbourhood_group      neighbourhood       price  page_id  line_id
0                 Bronx            Belmont   29.000000        1        1
1                 Bronx  Claremont Village   67.000000        1        2
2                 Bronx          Concourse   40.000000        1        3
3                 Bronx           Edenwald   90.000000        1        4
4                 Bronx           Edenwald   80.000000        1        5
..                  ...                ...         ...      ...      ...
804       Staten Island        South Beach  100.000000       30        4
805       Staten Island         St. George  130.000000       30        5
806       Staten Island          Todt Hill  135.000000       30        6
807       Staten Island      Tompkinsville   59.000000       30        7
808       Staten Island               MEAN   89.428571       30        8

[809 rows x 5 columns]

# prompt: Responda a seguinte pergunta em portguês:
# Considere as fases do processo CRISP DM. Relacione ao menos 6 tarefas presentes na preparação dos dados.

A fase de Preparação de Dados no CRISP-DM é crucial para garantir a qualidade e a relevância dos dados para a modelagem.  Aqui estão seis tarefas comuns nessa fase, com exemplos de como elas podem ser realizadas no contexto do código Python fornecido:

1. **Limpeza de Dados:** Trata-se de corrigir ou remover erros, inconsistências e valores ausentes nos dados.  No código, a linha `df_with_taxes['value'] = df_with_taxes['value'].replace(np.nan, 0)` demonstra essa tarefa, substituindo valores NaN (Not a Number) por zero na coluna 'value'.  Outras ações podem incluir remover linhas com dados faltantes ou substituir valores ausentes pela média ou mediana da coluna.

2. **Tratamento de Valores Ausentes:**  Como mencionado acima, a substituição de valores ausentes (NaN) por zero é uma estratégia.  Outra abordagem seria usar a mediana, por exemplo: `df['price'].fillna(df['price'].median(), inplace=True)`.  A escolha da estratégia depende da análise dos dados e do problema de negócio.  No código fornecido, o tratamento de valores faltantes está presente em diversas etapas.

3. **Transformação de Dados:**  Essa etapa envolve a conversão dos dados para um formato adequado para análise.  Um exemplo no código é a conversão dos valores da coluna 'price' para *float*: `df_aux.price = df_aux.price.astype(float)`.  Outras transformações podem incluir a normalização ou padronização de variáveis.

4. **Redução de Dimensionalidade:**  Se o conjunto de dados tem muitas variáveis, pode ser necessário reduzir o número de dimensões para melhorar o desempenho dos modelos e simplificar a análise.  No código, não há redução de dimensionalidade explícita, mas isso poderia ser feito usando técnicas como PCA (Principal Component Analysis) ou seleção de recursos.

5. **Integração de Dados:**  Pode ser necessário combinar dados de diferentes fontes. O código exemplifica isso ao unir dois dataframes com a função `merge`: `df_with_taxes = df.merge(df_taxes, how='left', on='id')`.

6. **Criação de novas variáveis (Feature Engineering):**  Uma etapa essencial para melhorar a qualidade dos modelos.  No código, a criação da coluna `df_word['word']` é um exemplo de *feature engineering*, categorizando as descrições dos anúncios com base na presença de certas palavras.  A criação de novas colunas derivadas das já existentes é fundamental para extrair informações mais relevantes para a modelagem.


Em resumo, a preparação de dados no CRISP-DM é um processo iterativo que envolve várias etapas. As tarefas listadas acima são apenas alguns exemplos, e a escolha das técnicas e das estratégias depende do contexto da análise e do problema a ser resolvido. O código fornecido exemplifica algumas dessas técnicas, mas o processo de preparação de dados é mais abrangente e exige ajustes específicos de acordo com o dataset.
